In [ ]:
import mysql.connector
import pandas as pd

from mysql.connector import errorcode

# Connect to MySQL
try:
    connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="password",
    database="timperio"
)
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Something is wrong with your username/password")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
    else:
        print(err)

cursor = connection.cursor()


In [ ]:
# Load the data from Excel and convert the date format
df_sales = pd.read_csv('CSV_Sales_Data_updated_Sep 2023.csv')
df_sales['Sale Date'] = pd.to_datetime(df_sales['Sale Date'], format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
df_sales = df_sales.replace({float("NaN"): None})

In [ ]:
# Insert the data into the MySQL table
for index, row in df_sales.iterrows():
    sql = """
    INSERT INTO sales (row_no, sale_date, sale_type, digital, customer_id, zip_code, 
                          shipping_method, product, variant, quantity, price, product_price) 
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    values = (
        row['Row No.'],
        row['Sale Date'],
        row['Sale Type'],
        row['Digital'],
        row['Customer ID'],
        row['ZipCode'],
        row['Shipping Method'],
        row['Product'],
        row['Variant'],
        row['Quantity'],
        row['Price'],
        row['Product Price']
    )
    cursor.execute(sql, values)

In [ ]:
# Commit and close
connection.commit()
cursor.close()
connection.close()